In [37]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.least_confidence import LeastConfidence
from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework1 import Framework1
import framework
import torch.nn as nn
import time
from torch.utils.data import DataLoader
from scipy.spatial.distance import jensenshannon
import al_methods
import os 

In [3]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },

            'svhn':
                {
                 'n_class':10,
                'channels':3,
                'size': 32,
                'transform_tr': transforms.Compose([ 
                                    transforms.RandomCrop(size = 32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4377, 0.4438, 0.4728), (0.1980, 0.2010, 0.1970))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.4377, 0.4438, 0.4728), 'std': (0.1980, 0.2010, 0.1970)},
                },
            'cifar10':
                {
                 'n_class':10,
                 'channels':3,
                 'size': 32,
                 'transform_tr': transforms.Compose([
                                    transforms.RandomCrop(size = 32, padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))]),
                 'loader_tr_args':{'batch_size': 256, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 512, 'num_workers': 8},
                 'normalize':{'mean': (0.4914, 0.4822, 0.4465), 'std': (0.2470, 0.2435, 0.2616)},
                 },


            'cifar100': 
               {
                'n_class':100,
                'channels':3,
                'size': 32,
                'transform_tr': transforms.Compose([
                                transforms.RandomCrop(size = 32, padding=4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))]),
                'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))]),
                'loader_tr_args':{'batch_size': 2048, 'num_workers': 4},
                'loader_te_args':{'batch_size': 512, 'num_workers': 8},
                'normalize':{'mean': (0.5071, 0.4867, 0.4408), 'std': (0.2675, 0.2565, 0.2761)},
                }
        }

In [2]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [4]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [5]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=50 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=1 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)


600 49 600


In [6]:

model=Model('resnet50').get_model()
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
model.conv1 = torch.nn.Conv2d(args_pool['mnist']['channels'], 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, args_pool['mnist']['n_class'])  # Modifier la couche de classification pour 10 classes
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file 
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

array([False, False, False, ..., False, False, False])

In [30]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.framework='framwork1'
        self.optimizer='SGD'
        self.save_model=False
        self.ALstrat='LeastConfidence'
        self.SSLstrat='fixmatch'

In [26]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [27]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

In [28]:
framework_1= Framework1(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [35]:
print(f' Sratgey for active learning{args.ALstrat} and strategy for semi-supervised learning used {args.SSLstrat}')
stratAl=LeastConfidence(framework_1.X_tr, framework_1.Y_tr, framework_1.X_te, framework_1.Y_te, framework_1.idxs_lb, framework_1.net, framework_1.handler, framework_1.args,framework_1.n_pool,framework_1.device)
stratSSL=fixmatch(framework_1.X_tr, framework_1.Y_tr, framework_1.X_te, framework_1.Y_te, framework_1.idxs_lb, framework_1.net, framework_1.handler, framework_1.args,framework_1.n_pool,framework_1.device,framework_1.predict,framework_1.g)


framework_1.train(alpha=2e-3,n_epoch=10)

test_acc=framework_1.predict(framework_1.X_te,framework_1.Y_te)
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

for rd in range(0, NUM_ROUND):
    
    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(framework_1.n_pool)[framework_1.idxs_lb])
        if NUM_QUERY > int(nEnd*framework_1.n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*framework_1.n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = stratAl.query(NUM_QUERY)
        q_idxs = output
        framework_1.idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_1.update(framework_1.idxs_lb)
        if hasattr(stratAl, 'train'):
        
            best_test_acc=stratAl.train(alpha=2e-3, n_epoch=10)
        else: best_test_acc = framework_1.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(framework_1.n_pool)[framework_1.idxs_lb])
        if NUM_QUERY > int(nEnd*framework_1.n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*framework_1.n_pool/100) - labeled
            
        # query
        ts = time.time()

        output = stratSSL.query(NUM_QUERY)
        q_idxs = output
        framework_1.idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        framework_1.update(framework_1.idxs_lb)
        best_test_acc = stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc



 Sratgey for active learningLeastConfidence and strategy for semi-supervised learning used fixmatch
Let's use 1 GPUs!
[Batch=000] [Loss=1.25]

==>>[2023-08-24 09:34:14] [Epoch=000/010] [framwork1(LeastConfidence+fixmatch) Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=1.13]

==>>[2023-08-24 09:34:15] [Epoch=001/010] [framwork1(LeastConfidence+fixmatch) Need: 00:00:14] [LR=0.1000] [Best : Test Accuracy=0.44, Error=0.56]
[Batch=000] [Loss=1.17]

==>>[2023-08-24 09:34:17] [Epoch=002/010] [framwork1(LeastConfidence+fixmatch) Need: 00:00:12] [LR=0.1000] [Best : Test Accuracy=0.44, Error=0.56]
[Batch=000] [Loss=1.11]

==>>[2023-08-24 09:34:19] [Epoch=003/010] [framwork1(LeastConfidence+fixmatch) Need: 00:00:11] [LR=0.1000] [Best : Test Accuracy=0.53, Error=0.47]
[Batch=000] [Loss=0.82]

==>>[2023-08-24 09:34:20] [Epoch=004/010] [framwork1(LeastConfidence+fixmatch) Need: 00:00:09] [LR=0.1000] [Best : Test Accuracy=0.57, Error=0.43]
[Batch=000] [Loss=0.77

In [36]:
acc

array([0.80019999, 0.88270003, 0.85420001, 0.6972    , 0.81489998,
       0.89899999, 0.94929999, 0.97670001, 0.97180003, 0.986     ,
       0.98000002, 0.98750001, 0.9867    , 0.99169999, 0.9817    ,
       0.9914    , 0.98689997, 0.99260002, 0.9874    , 0.99309999,
       0.98809999, 0.99309999, 0.98790002, 0.99349999, 0.99059999,
       0.9928    , 0.98830003, 0.99309999, 0.98650002, 0.99299997,
       0.99070001, 0.99330002, 0.98940003, 0.99400002, 0.9874    ,
       0.9932    , 0.99019998, 0.99269998, 0.98580003, 0.9932    ,
       0.98760003, 0.9928    , 0.99010003, 0.99370003, 0.98729998,
       0.99360001, 0.98820001, 0.9914    , 0.9878    , 0.        ])

In [39]:
folder_result_acc='results'
# out_file = os.path.join(args.save_path, args.save_file)
if not os.path.exists(folder_result_acc):
    os.mkdir(folder_result_acc)
    print(f"Folder '{folder_result_acc}' created succesfuly.")
file_path=os.path.join(folder_result_acc,args.framework+"("+args.ALstrat+args.SSLstrat+")")
np.save(file_path,acc)

In [40]:
np.load("results/framwork1(LeastConfidencefixmatch).npy")

array([0.80019999, 0.88270003, 0.85420001, 0.6972    , 0.81489998,
       0.89899999, 0.94929999, 0.97670001, 0.97180003, 0.986     ,
       0.98000002, 0.98750001, 0.9867    , 0.99169999, 0.9817    ,
       0.9914    , 0.98689997, 0.99260002, 0.9874    , 0.99309999,
       0.98809999, 0.99309999, 0.98790002, 0.99349999, 0.99059999,
       0.9928    , 0.98830003, 0.99309999, 0.98650002, 0.99299997,
       0.99070001, 0.99330002, 0.98940003, 0.99400002, 0.9874    ,
       0.9932    , 0.99019998, 0.99269998, 0.98580003, 0.9932    ,
       0.98760003, 0.9928    , 0.99010003, 0.99370003, 0.98729998,
       0.99360001, 0.98820001, 0.9914    , 0.9878    , 0.        ])